Connected to Python 3.11.5

In [3]:
import yolov5
import torch
import random
from pathlib import Path
import paho.mqtt.client as mqtt
import mysql.connector
from datetime import datetime
import time

# MySQL Configuration
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'WillofD@30',
    'database': 'satwa'
}

# Waste Class Angle Mapping
class_angle_mapping = {
    "biological": 0,
    "brown glass": 30,
    "white glass": 60,
    "trash": 90,
    "shoes": 120,
    "plastic": 150,
    "paper": 180,
    "clothes": 210,
    "metal": 240,
    "cardboard": 270,
    "green glass": 300,
    "unknown": 330
}

# Waste Class Weight Ranges (in grams)
class_weight_range = {
    "biological": (50, 200),
    "brown glass": (100, 500),
    "white glass": (100, 500),
    "trash": (50, 300),
    "shoes": (300, 1000),
    "plastic": (50, 250),
    "paper": (20, 200),
    "clothes": (200, 800),
    "metal": (500, 2000),
    "cardboard": (100, 600),
    "green glass": (100, 500),
    "unknown": (50, 300)
}

# Connect to MySQL
db = mysql.connector.connect(**db_config)
cursor = db.cursor()

# Drop existing waste_weights table and create a new one
cursor.execute("DROP TABLE IF EXISTS waste_weights")
db.commit()

create_weight_table = """
CREATE TABLE waste_weights (
    waste_class VARCHAR(255) PRIMARY KEY,
    total_weight INT DEFAULT 0
);
"""
cursor.execute(create_weight_table)
db.commit()

# Initialize waste_weights table with 12 classes and zero weight
for waste_class in class_angle_mapping.keys():
    cursor.execute("INSERT INTO waste_weights (waste_class, total_weight) VALUES (%s, %s)", (waste_class, 0))
db.commit()

# Load YOLOv5 Model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = yolov5.load('keremberke/yolov5m-garbage')
model.to(device)
model.conf = 0.25
model.iou = 0.45

# Get class names
class_names = model.names

# Base directory for images
base_dir = r"C:\Users\Lenovo\Downloads\archive (3)\garbage_classification"
base_path = Path(base_dir)
subfolders = sorted([f for f in base_path.rglob('*') if f.is_dir()])

def process_random_image(subfolder, payload_value):
    image_extensions = ('.jpg', '.jpeg', '.png')
    images = [f for f in subfolder.iterdir() if f.is_file() and f.suffix.lower() in image_extensions]
    
    if not images:
        return None, None
    
    random_image = random.choice(images)
    print(f"\nProcessing Image: {random_image}")

    results = model(str(random_image), size=640)
    predictions = results.pred[0]
    
    predicted_class = "unknown"
    if len(predictions) > 0:
        scores = predictions[:, 4]
        top_idx = scores.argmax()
        top_cat = int(predictions[top_idx, 5])
        predicted_class = class_names[top_cat] if top_cat < len(class_names) else "unknown"
        
        print(f"Classified as: {predicted_class} from {random_image.name}")
    else:
        print("No objects detected.")

    timestamp = datetime.now()
    insert_query = "INSERT INTO garbage_classification (timestamp, payload_value, predicted_class) VALUES (%s, %s, %s)"
    cursor.execute(insert_query, (timestamp, payload_value, predicted_class))
    db.commit()
    
    return predicted_class, random_image.name

def update_waste_weight(waste_class):
    if waste_class in class_weight_range:
        weight = random.randint(*class_weight_range[waste_class])
        print(f"Generated weight for {waste_class}: {weight}g")
        
        cursor.execute("SELECT total_weight FROM waste_weights WHERE waste_class = %s", (waste_class,))
        result = cursor.fetchone()
        
        new_weight = (result[0] + weight) if result else weight
        cursor.execute("UPDATE waste_weights SET total_weight = %s WHERE waste_class = %s", (new_weight, waste_class))
        
        db.commit()
        print(f"Updated total weight for {waste_class}: {new_weight}g")
        return weight
    return None

def on_message(client, userdata, msg):
    try:
        value = int(msg.payload.decode())
        print(f"\nReceived MQTT value from IoT sensor: {value}")
        
        folder_idx = (value - 1) // 10  
        if folder_idx < len(subfolders):
            selected_folder = subfolders[folder_idx]
            predicted_class, image_name = process_random_image(selected_folder, value)
            angle = class_angle_mapping.get(predicted_class, class_angle_mapping["unknown"])
            
            print(f"Servo motor rotated to {angle}° for detected class: {predicted_class} from {image_name}")
            
            client.publish("garbage/servo_angle", str(angle))
            print(f"Published angle {angle}° to MQTT topic 'garbage/servo' for servo rotation.")
            print(f"Servo meter now rotated by {angle}°.")
            
            weight = update_waste_weight(predicted_class)
            if weight is not None:
                client.publish("garbage/weight", str(weight))
                print(f"Published weight {weight}g to MQTT topic 'garbage/weight'.")
    except ValueError:
        print("Invalid payload received.")

broker = "test.mosquitto.org"
topic = "garbage/index"  # MQTT topic receiving values from IoT sensors

client = mqtt.Client()
client.on_message = on_message
client.connect(broker, 1883, 60)
client.subscribe(topic)

print(f"Subscribed to MQTT topic '{topic}' for receiving IoT sensor values.")
print("Waiting for messages...")

try:
    client.loop_forever()
except KeyboardInterrupt:
    cursor.close()
    db.close()
    client.disconnect()
    print("Disconnected from MQTT and MySQL. System shutting down.")


YOLOv5  2025-2-24 Python-3.11.5 torch-2.6.0+cpu CPU

Fusing layers... 
Model summary: 212 layers, 20873139 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_2504\18704986.py:161: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Subscribed to MQTT topic 'garbage/index' for receiving IoT sensor values.
Waiting for messages...

Received MQTT value from IoT sensor: 22

Processing Image: C:\Users\Lenovo\Downloads\archive (3)\garbage_classification\cardboard\cardboard869.jpg


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\yolov5\models\common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Classified as: cardboard from cardboard869.jpg
Servo motor rotated to 270° for detected class: cardboard from cardboard869.jpg
Published angle 270° to MQTT topic 'garbage/servo' for servo rotation.
Servo meter now rotated by 270°.
Generated weight for cardboard: 173g
Updated total weight for cardboard: 173g
Published weight 173g to MQTT topic 'garbage/weight'.

Received MQTT value from IoT sensor: 22

Processing Image: C:\Users\Lenovo\Downloads\archive (3)\garbage_classification\cardboard\cardboard799.jpg


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\yolov5\models\common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Classified as: cardboard from cardboard799.jpg
Servo motor rotated to 270° for detected class: cardboard from cardboard799.jpg
Published angle 270° to MQTT topic 'garbage/servo' for servo rotation.
Servo meter now rotated by 270°.
Generated weight for cardboard: 149g
Updated total weight for cardboard: 322g
Published weight 149g to MQTT topic 'garbage/weight'.

Received MQTT value from IoT sensor: 69

Processing Image: C:\Users\Lenovo\Downloads\archive (3)\garbage_classification\paper\paper510.jpg


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\yolov5\models\common.py:709: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Classified as: paper from paper510.jpg
Servo motor rotated to 180° for detected class: paper from paper510.jpg
Published angle 180° to MQTT topic 'garbage/servo' for servo rotation.
Servo meter now rotated by 180°.
Generated weight for paper: 135g
Updated total weight for paper: 135g
Published weight 135g to MQTT topic 'garbage/weight'.
Disconnected from MQTT and MySQL. System shutting down.
